In [ ]:
# i dont know what it is , i will keep it here in case i found it necessary in my code

def create_simple_convnet_model(*, input_size, output_size,  verbose=False, **kwargs):
                         
    # Convolutional Network, ........................
    model = keras.Sequential()

    #.. 1st cnn, layer
    model.add(keras.layers.Conv2D(
        filters=kwargs['Conv2D_1__filters'], 
        kernel_size=kwargs['Conv2D_1__kernel_size'], 
        strides=kwargs['Conv2D_1__stride'],
        activation=kwargs['Conv2D_1__activation'], 
        input_shape=input_size
        ))

    #.. maxpool 1.
    model.add(keras.layers.MaxPool2D(pool_size=kwargs['MaxPool2D_1__pool_size']))

    #.. 2nd cnn layer, 
    model.add(keras.layers.Conv2D(
        filters=kwargs['Conv2D_2__filters'], 
        kernel_size=kwargs['Conv2D_2__kernel_size'], 
        strides=kwargs['Conv2D_2__stride'],
        activation=kwargs['Conv2D_2__activation'], 
        ))

    #.. maxpool 2, 
    model.add(keras.layers.MaxPool2D(pool_size=kwargs['MaxPool2D_2__pool_size']))

    
    # flatten the results, .........................
    model.add(keras.layers.Flatten())
    
    
    # dense nn, ....................................
    
    #.. First hidden layer
    model.add(Dense(
        units=kwargs['h1_unit_size'], 
        activation=kwargs["h1_activation"], 
        kernel_initializer=initializers.VarianceScaling(scale=2.0, seed=0)
    ))
    model.add(tf.keras.layers.Dropout(kwargs["h1_Dropout"]))
    
    #.. Output layer
    model.add(Dense( 
        units=output_size, 
        activation=kwargs["out_activation"],
        kernel_regularizer=tf.keras.regularizers.l2(0.001),
        kernel_initializer=initializers.VarianceScaling(scale=1.0, seed=0)
    ))

    
    # Print network summary
    if verbose==True:
        print(model.summary())
    else:
        pass

    
    # Define Loss Function and Trianing Operation 
    model.compile(
        optimizer= kwargs["optimizer"],
        loss= losses.sparse_categorical_crossentropy,
        metrics= kwargs["metrics"] # even one arg must be in the list
    )
       
    return model                  
                         
                         
                         
                         

In [ ]:
grid = ParameterGrid({
    "random_state":[0], # used to divide train,valid datasets, 
    "train_test_split__train_size":[0.7],
    # -- generators, ........................
    "generator__batch_size": [20],
    "generator__target_size" :[(299, 299)], # tuple, for each image x,y dimension in pixels, 
    "generator__validation_split" : [0.3], # caution its opposite then in train_test_split__train_size"
    # -- conv model, ........................
    "model":["two_layers"],
    'Conv2D_1__filters': [64], 
    'Conv2D_1__kernel_size': [5], 
    'Conv2D_1__stride': [2], 
    'Conv2D_1__activation' : ['relu'], 
    'MaxPool2D_1__pool_size': [2], 
    'Conv2D_2__filters': [64], 
    'Conv2D_2__kernel_size': [3], 
    'Conv2D_2__stride': [1], 
    'Conv2D_2__activation' : ["relu"], 
    'MaxPool2D_2__pool_size': [2], 
    # -- dense nn, ...........................
    "h1_unit_size":[24],
    "h1_Dropout" : [0],
    "h1_activation": ["relu"],
    "out_activation":["softmax"],
    "optimizer":["Adam"],
    "metrics": [["acc"]],
    # -- training, ...........................
    "EarlyStopping__patience": [6],
    "fit_generator__epoch": [2]     
    
})

params